In [ ]:
import keras as k
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

import numpy as np
from random import randint
from sklearn.preprocessing import MinMaxScaler

<h3> Creating Example Data to Processing </h3>

In [ ]:
train_labels = []
train_samples = []

<h4>Example Data : </h4>
<ul>
    <li>An experimental drug was tested on individuals from ages 13 to 65</li>
    <li>The trial had 2100 participants. Half were under 65 years old, half were over 65 years old.</li>
    <li>95% of patients 65 or older experienced side effects.</li>
    <li>95% of patients under 65 experienced no side effects.</li>
</ul>


In [ ]:
for i in range (1000):
    random_younger = randint(13,64)
    train_samples.append(random_younger)
    train_labels.append(0)
    
    random_older = randint(65,100)
    train_samples.append(random_older)
    train_labels.append(1)
    
for i in range(50):
    random_younger = randint(13,64)
    train_samples.append(random_younger)
    train_labels.append(1)
    
    random_older = randint(65,100)
    train_samples.append(random_older)
    train_labels.append(0)

In [ ]:
# print raw data
for i in train_samples:
    print(i)

In [ ]:
train_labels = np.array(train_labels)
train_samples = np.array(train_samples)

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_train_samples = scaler.fit_transform((train_samples).reshape(-1,1))

In [ ]:
#print scaled data
for i in scaled_train_samples:
    print(i)

<h3> Build a Simple Model </h3>

In [ ]:
model = Sequential([
    Dense(16,input_shape=(1,), activation="relu"),
    Dense(32, activation="relu"),
    Dense(2, activation="softmax")
])

In [ ]:
model.summary()

<h3> Train Model </h3>

In [ ]:
model.compile(Adam(lr=.0001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.fit(scaled_train_samples, train_labels, validation_split=0.1, batch_size=10, epochs=20, shuffle=True, verbose=2)

<h2> Build a Validation Data </h2>
<p> valid_set = [(sample, label), (sample, label), ... , (sample, label)] </p>
<p> model.fit(scaled_train_samples, train_labels, validation_data=valid_set, batch_size=10, epochs=20, shuffle=True, verbose=2) </p>

<h3> Prediction </h3>

In [ ]:
# Creating Test Data
test_labels = []
test_samples = []

In [ ]:
for i in range(10):
    #the 5% of younger individuals who did experience side effects
    random_younger = randint(13,64)
    test_samples.append(random_younger)
    test_labels.append(1)
    
    #the 5% of older individuals who did not experience side effects
    random_older = randint(64,100)
    test_samples.append(random_older)
    test_labels.append(0)

for i in range(200):
    #the 95% of younger individuals who did experience side effects
    random_younger = randint(13,64)
    test_samples.append(random_younger)
    test_labels.append(1)
    
    #the 95% of older individuals who did not experience side effects
    random_older = randint(64,100)
    test_samples.append(random_older)
    test_labels.append(0)
    

In [ ]:
test_labels = np.array(train_labels)
test_samples = np.array(train_samples)

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_test_samples = scaler.fit_transform((test_samples).reshape(-1,1))

In [ ]:
predictions = model.predict(scaled_test_samples, batch_size=10, verbose=0)

In [ ]:
for i in predictions:
    print(i)

In [ ]:
rounded_predictions = model.predict_classes(scaled_test_samples, batch_size=10, verbose=0)

In [ ]:
for i in rounded_predictions:
    print(i)

<h3> Create Confusion Matrix </h3>

In [ ]:
%matplotlib inline
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

In [ ]:
cm = confusion_matrix(test_labels, rounded_predictions)

In [ ]:
def plotCm(cm,
          classes,
          normalize=False,
          title='confusion_matrix',
          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting 'normalize=True'.
    """
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes,rotation=45)
    plt.yticks(tick_marks, classes)
    
    if normalize:
        cm = cm.astype('float')/ cm.sum(axis=1)[:,np.newaxis]
        print("Normalized Confusion MAtrix")
    else:
        print("COnfusion Matrix, Without Normalization")
        
    print(cm)
    
    thresh = cm.max() / 2.
    
    for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i,j],
                horizontalalignment = 'center',
                color='white' if cm[i, j] > thresh else "black")
        
    plt.tight_layout()
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    

In [ ]:
cm_plot_labels = ['no_side_effects', 'half_side_effects']

plotCm(cm, cm_plot_labels, title='Confusion Matrix')

<h2> Save And Load Model </h2>

<h6> The Save functions saves : </h6>
<ul>
    <li>The architecture of the model allowing to re-create the model</li>
    <li>The weights of the model</li>
    <li>The training configurations(loss, optimizer)</li>
    <li>The state of the optimizer, allowing to resume training exactly where you are left off</li>
</ul>

In [ ]:
#Saving Model
model.save('medical_trial_model.h5')

#Load Model
from keras.models import load_model
new_model = load_model('medical_trial_model.h5')

In [ ]:
new_model.summary()

In [ ]:
new_model.get_weights()

In [ ]:
new_model.optimizer

<h3>Model To .Json</h3>

if you only need tosave the architecture of a model, and not its weights or its training configuration, you can use the following function to save the architecture only.

In [ ]:
# Save as Json
json_string = model.to_json()

# Save as Yaml
#yaml_strings = model.to_yaml()


In [ ]:
json_string

In [ ]:
# model reconstruction from json

from keras.models import model_from_json

model_architecture = model_from_json(json_string)

In [ ]:
model_architecture.summary()

<h3>Model Save Weights</h3>

<p> If you  only need to save the weights of a model, you can use the following function save the weights only </p>

In [ ]:
model.save_weights('model_weights.h5')

In [ ]:
model2 = Sequential([
    Dense(16, input_shape=(1,), activation="relu"),
    Dense(32, activation="relu"),
    Dense(2, activation="softmax")
])

In [ ]:
model2.load_weights('model_weights.h5')